In [295]:
import json
from enum import Enum
from gurobipy import Model, GRB, quicksum, max_
import numpy as np
import pandas as pd
from random import randint


In [296]:
class DATASET(Enum):
    TOY = "toy"
    MEDIUM = "medium"
    LARGE = "large"
    GEN_1 = "generated_1"
    GEN_2 = "generated_2"
    GEN_3 = "generated_3"

In [297]:
def load_data(name):
    """name must be an instance of DATASET like DATASET.TOY for example"""
    if not isinstance(name, DATASET):
        raise TypeError("direction must be an instance of DATASET Enum")
    with open(f"../data/{name.value}_instance.json", "r") as f:
        data = json.load(f)
    return data


def get_dims(data):
    return (
        len(data["staff"]),
        data["horizon"],
        len(data["qualifications"]),
        len(data["jobs"]),
    )

def get_qualification_index(list_qualifications, qualification): # qualification is "A", "B", "C" ...
    return list_qualifications.index(qualification)

In [298]:
def init_model():
    m = Model("Project modelling")
    return m

In [299]:
def create_decision_variables(model, n_staff, horizon, n_qualifs, n_jobs):
    X = model.addVars(n_staff, horizon, n_qualifs, n_jobs, vtype=GRB.BINARY, name="assignements")
    J = model.addVars(n_jobs, vtype=GRB.BINARY, name="completion")
    E_D = model.addVars(n_jobs, lb=0, ub=horizon-1, vtype=GRB.INTEGER, name="end_dates")
    L = model.addVars(n_jobs, lb=0, ub=horizon + 1, vtype=GRB.INTEGER, name="n_days_late")
    # for max(days spent on project) variable
    S_D = model.addVars(n_jobs, lb=0, ub=horizon-1, vtype=GRB.INTEGER, name="start_dates")
    spans = model.addVars(n_jobs, lb=0, ub=horizon, vtype=GRB.INTEGER, name="spans")
    max_days = model.addVar(lb=0, ub=horizon, vtype=GRB.INTEGER, name="max_jobs")
    # for max(projects affected) variable 
    n_jobs_per_person = model.addVars(n_staff, lb=0, ub=n_jobs, vtype=GRB.INTEGER, name="n_jobs_per_person")
    jobs_worked_on_by_person = model.addVars(n_staff, n_jobs, vtype=GRB.BINARY, name="jobs_worked_on_by_person")
    n_worked_days_per_job_and_person = model.addVars(n_staff, n_jobs,lb=0, ub=horizon, vtype=GRB.INTEGER, name="n_worked_days_per_job_and_person")
    max_jobs = model.addVar(lb=0, ub=n_jobs, vtype=GRB.INTEGER, name="max_jobs")
    return model, X, J, S_D, E_D, L, n_jobs_per_person, jobs_worked_on_by_person, n_worked_days_per_job_and_person, max_jobs, max_days, spans


In [300]:
def add_constraints_start_dates(model, X, S_D, n_staff, horizon, n_qualifs, n_jobs):
    # model.addConstrs(
    #     (X[i, j, k, l] == 1) >> (X[i, j, k, l] * j >= S_D[l]) 
    #     for i in range(n_staff)
    #     for j in range(horizon)
    #     for k in range(n_qualifs)
    #     for l in range(n_jobs)
    # )
    model.addConstrs(
        S_D[l] <= X[i, j, k, l] * j + (1 - X[i, j, k, l]) * horizon
        for i in range(n_staff)
        for j in range(horizon) 
        for k in range(n_qualifs)
        for l in range(n_jobs)
    )

In [301]:
def add_constraints_end_dates(model, X, E_D, n_staff, horizon, n_qualifs, n_jobs):
    model.addConstrs(
        X[i, j, k, l] * j <= E_D[l] 
        for i in range(n_staff)
        for j in range(horizon)
        for k in range(n_qualifs)
        for l in range(n_jobs)
    )

In [302]:
def add_constraints_lateness(model, E_D, L, jobs, n_staff, horizon, n_qualifs, n_jobs):
    model.addConstrs(
        E_D[l] +1 - jobs[l]["due_date"] <= L[l] for l in range(n_jobs)
    )

In [303]:
def add_constraints_worked_days_below_required_days(model, X, jobs, qualifications, n_staff, horizon, n_jobs):
    model.addConstrs(quicksum(X[i,j,get_qualification_index(qualifications, k),l] for i in range(n_staff) for j in range(horizon)) <= jobs[l]["working_days_per_qualification"][k] 
                     for l in range(n_jobs) 
                     for k in jobs[l]["working_days_per_qualification"].keys())
    

In [304]:
def add_constraints_worked_days_above_required_days(model, X, J, jobs, qualifications, n_staff, horizon, n_jobs):
    model.addConstrs(quicksum(X[i,j,get_qualification_index(qualifications, k),l] for i in range(n_staff) for j in range(horizon)) >= J[l]* jobs[l]["working_days_per_qualification"][k] 
                     for l in range(n_jobs) 
                     for k in jobs[l]["working_days_per_qualification"].keys())

    # for l in range(n_jobs): 
    #     for quali,njk in data["jobs"][l]["working_days_per_qualification"].items():
    #         model.addConstr(J[l]*njk <= quicksum([X[i,j,get_qualification_index(qualifications, quali),l] for i in range(n_staff) for j in range(horizon)]))

In [305]:
def in_qualification(data, i, k):
    return data["qualifications"][k] in data["staff"][i]["qualifications"]


def add_qualification_constraints(model, n_staff, horizon, n_qualifs, n_jobs, X, data):
    model.addConstrs(
        X[i, j, k, l] == 0
        for i in range(n_staff)
        for j in range(horizon)
        for k in range(n_qualifs)
        for l in range(n_jobs)
        if not in_qualification(data, i, k)
    )


def in_vacation(i, j, data):
    data = {l: data["staff"][l] for l in range(len(data["staff"]))}
    data = data[i]["vacations"]
    return j in data


def add_vacation_constraints(model, n_staff, horizon, n_qualifs, n_jobs, X, data):
    model.addConstrs(
        X[i, j, k, l] == 0
        for i in range(n_staff)
        for j in range(horizon)
        for k in range(n_qualifs)
        for l in range(n_jobs)
        if in_vacation(i, j, data)
    )

In [306]:
def add_constraints_employees_working_only_one_day(model, X, J, data, n_staff, n_jobs, horizon, n_qualifs):
    model.addConstrs(quicksum(X[i,j,k,l] for l in range(n_jobs) for k in range (n_qualifs)) <= 1 
                     for i in range(n_staff) 
                     for j in range(horizon))

In [307]:
def add_constraints_n_worked_days_per_jobs_person(model, X, n_worked_days_per_job_and_person, n_staff, n_jobs, horizon, n_qualifs):
    model.addConstrs( n_worked_days_per_job_and_person[i, l] == quicksum( X[i,j,k,l] for j in range(horizon) for k in range(n_qualifs)) 
        for i in range(n_staff) 
        for l in range(n_jobs)
    )

def add_constraints_jobs_worked_on_by_person(model, jobs_worked_on_by_person, n_worked_days_per_job_and_person, n_staff, n_jobs):
    model.addConstrs((jobs_worked_on_by_person[i, l] == 0) >> (n_worked_days_per_job_and_person[i,l] == 0) 
        for i in range(n_staff) 
        for l in range(n_jobs)
    ) 
    # model.addConstrs((jobs_worked_on_by_person[i, l] <= n_worked_days_per_job_and_person[i,l]) 
    #     for i in range(n_staff) 
    #     for l in range(n_jobs)
    # ) 
    model.addConstrs((jobs_worked_on_by_person[i, l] == 1) >> (n_worked_days_per_job_and_person[i,l] >= 1) 
        for i in range(n_staff) 
        for l in range(n_jobs)
    )
    # model.addConstrs((n_worked_days_per_job_and_person[i, l] <= jobs_worked_on_by_person[i,l]*n_jobs) 
    #     for i in range(n_staff) 
    #     for l in range(n_jobs)
    # ) 

def add_constraints_n_jobs_per_person(model, n_jobs_per_person, jobs_worked_on_by_person, n_staff, n_jobs):
    model.addConstrs(n_jobs_per_person[i] == quicksum( jobs_worked_on_by_person[i, l] for l in range(n_jobs) ) for i in range(n_staff))

def add_constraint_max_jobs(model, max_jobs, n_jobs_per_person, jobs_worked_on_by_person, n_staff):
    model.addConstr(max_jobs == max_([n_jobs_per_person[i] for i in range(n_staff)]))

In [308]:
def add_constraint_spans(model, spans, E_D, S_D, n_jobs):
    model.addConstrs((spans[l] == (E_D[l] - S_D[l])+1) for l in range(n_jobs))

def add_constraint_max_days(model, max_days, spans, n_jobs):
    for l in range(n_jobs):
        model.addConstr(max_days >= spans[l]) 
    # model.addConstr(max_days == max_([spans[l] for l in range(n_jobs)]))

In [309]:
def add_profit_as_first_objective(model, J, L, jobs):
    model.setObjective(
        quicksum( (J[index_job] * job["gain"] - job["daily_penalty"] * L[index_job]) for index_job, job in enumerate(jobs)),
        GRB.MAXIMIZE
    )
    return model

## add pouieme des autres

In [310]:
def add_minimax_jobs_as_second_objective(model, max_jobs):
    model.setObjective(
        max_jobs,
        GRB.MINIMIZE
    )
    return model

In [311]:
def add_minimax_days_spent_as_third_objective(model, max_days):
    model.setObjective(
        max_days,
        GRB.MINIMIZE
    )
    return model

In [312]:
def add_mono_objective(model, J, L, jobs, max_days, max_jobs):
    profit = quicksum( (J[index_job] * job["gain"] - job["daily_penalty"] * L[index_job]) for index_job, job in enumerate(jobs))
    model.setObjective(
        10 * profit - max_days - max_jobs,
        GRB.MAXIMIZE
    )
    return model

In [313]:
# Importing data
data = load_data(DATASET.GEN_2)
n_staff, horizon, n_qualifs, n_jobs = get_dims(data)

# Instanciation du modèle
model = init_model()

# Création des variables : binaires dans X et J, entières de 0 à horizon + 3
model, X, J, S_D, E_D, L, n_jobs_per_person, jobs_worked_on_by_person, n_worked_days_per_job_and_person, max_jobs, max_days, spans = create_decision_variables(model, n_staff, horizon, n_qualifs, n_jobs)

# maj du modèle
model.update()

# Ajout des constraintes

add_constraints_employees_working_only_one_day(model, X ,J,data,n_staff,n_jobs,horizon, n_qualifs)
add_qualification_constraints(model, n_staff, horizon, n_qualifs, n_jobs, X, data)
add_vacation_constraints(model, n_staff, horizon, n_qualifs, n_jobs, X, data)

add_constraints_start_dates(model, X, S_D, n_staff, horizon, n_qualifs, n_jobs)
add_constraints_end_dates(model, X, E_D, n_staff, horizon, n_qualifs, n_jobs)
add_constraints_lateness(model, E_D, L, data["jobs"], n_staff, horizon, n_qualifs, n_jobs)

add_constraints_worked_days_below_required_days(model, X, data["jobs"], data["qualifications"], n_staff, horizon, n_jobs)
add_constraints_worked_days_above_required_days(model, X , J, data["jobs"], data["qualifications"], n_staff, horizon, n_jobs)

add_constraints_n_worked_days_per_jobs_person(model, X, n_worked_days_per_job_and_person, n_staff, n_jobs, horizon, n_qualifs)
add_constraints_jobs_worked_on_by_person(model, jobs_worked_on_by_person, n_worked_days_per_job_and_person, n_staff, n_jobs)
add_constraints_n_jobs_per_person(model, n_jobs_per_person, jobs_worked_on_by_person, n_staff, n_jobs)
add_constraint_spans(model, spans, E_D, S_D, n_jobs)

add_constraint_max_jobs(model, max_jobs, n_jobs_per_person, jobs_worked_on_by_person, n_staff)
add_constraint_max_days(model, max_days, spans, n_jobs)

# maj du modèle
model.update()

# Fonctions Objectifs

# model = add_minimax_jobs_as_second_objective(model, max_jobs)
# model = add_minimax_days_spent_as_third_objective(model, max_days)
# model = add_profit_as_first_objective(model, J, L, data["jobs"])

model = add_mono_objective(model, J, L, data["jobs"], max_days, max_jobs)

# maj du modèle
model.update()

# Paramétrage (mode mute)
model.params.outputflag = 0

# Résolution du PL
model.optimize()

print(round(model.ObjVal))
print(type(model.ObjVal))
print(round(max_days.X), "days")
print(round(max_jobs.X), "jobs")
# print(n_jobs_per_person, "n_jobs_per_person")
# print(max_jobs, "max_jobs")
# print(J, "completion")
# print("\n")
# print(E_D, "end_dates")
# print("\n")
# print(S_D, "start_dates")
# print(L, "late")
# print("\n")
# print(spans, "spans")


1274
<class 'float'>
3 days
3 jobs


In [314]:
## affichage solutions

def format_df(styler):
    color = []
    for i in range(n_jobs):
        color.append('#%06X' % randint(0, 0xFFFFFF))
    styler.applymap(lambda x: f"background-color: {color[x[1]]}" if x[0]!=-1 else "background-color: black")
    styler.format(lambda x: data["qualifications"][x[0]] if x[0]!=-1 else "")
    return styler

staffos = {i:[] for i in range(n_staff)}
for i in range(n_staff):
    for j in range(horizon):
        done = False
        for k in range(n_qualifs):
            for l in range(n_jobs):
                if round(X[(i, j, k, l)].X)==1:
                    staffos[i].append([k,l])
                    done = True
        if done==False:
            staffos[i].append([-1,-1])
            
df=pd.DataFrame(staffos).T
df.index=[data["staff"][i]["name"] for i in range(n_staff)]

    
df.style.pipe(format_df)

,0,1,2,3,4,5,6,7
Person1,,,,,Skill4,Skill4,Skill3,Skill1
Person2,Skill4,,Skill4,Skill2,,Skill2,Skill4,Skill4
Person3,Skill3,Skill3,Skill4,,Skill3,Skill3,,
Person4,Skill3,Skill3,,Skill3,,,,Skill3
Person5,Skill1,Skill1,Skill3,Skill3,Skill1,Skill3,Skill1,
Person6,,Skill1,Skill3,,,Skill3,,Skill1


In [315]:
# # Objective Function 1: Maximize Profits
# model.setObjective(sum(profit[j]*y[j] for j in projects) - sum(penalty[i,j]*x[i,j,k] for i in staff for j in qualifications for k in working_days), GRB.MAXIMIZE)

# # Objective Function 2: Minimize Number of Projects per Employee
# model.setObjective(sum(x[i,j,k] for i in staff for j in qualifications for k in working_days) , GRB.MINIMIZE)
# model.setObjective(sum(x[i,j,k] for i in staff for j in qualifications for k in working_days) , GRB.MINIMIZE)

# # Objective Function 3: Minimize Length of Longest Project
# model.setObjective(max(duration[j] for j in projects), GRB.MINIMIZE)